In [14]:
#install packages
!pip install numba==0.48
!pip install aif360==0.3.0rc0
!pip install BlackBoxAuditing

In [15]:
#import packages
import numpy as np
import pandas as pd

import BlackBoxAuditing

from aif360.algorithms.preprocessing import Reweighing, DisparateImpactRemover
from aif360.datasets import AdultDataset, StandardDataset, BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.postprocessing import EqOddsPostprocessing, RejectOptionClassification
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

from google.colab import files
import io

## Loading and Splitting Data

In [16]:
#read in the dataset
uploaded = files.upload()
heart = pd.read_csv(io.BytesIO(uploaded['heart.csv']))

heart.head()

Saving heart.csv to heart (1).csv


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [17]:
#split data into X and y and scale the X
X = heart[list(heart.columns)[0:-1]]
y = heart['target']

X_norm = normalize(X, norm='l2')

In [18]:
#split the dataset into train, val, test using the same seeds as the ADS
X_train, X_hold, y_train, y_hold = train_test_split(X_norm, y, test_size=0.33, random_state=101) 
X_val, X_test, y_val, y_test = train_test_split(X_hold, y_hold, test_size=0.5, random_state=42)

#PROBLEM: BY NORMALIZING IN THE MANNER IN WHICH HE DID, HE CHANGED THE INPUT SPACE OF FEATURES LIKE SEX
#WE NEED TO MAP THIS NEW COLUMN BACK TO A BINARY 0/1 COLUMN IN THE DATASET BEFORE FITTING THE DATASET

#resplitting data with same seed without normalizing (used to validate that 0 stayed 0 and 1 changed to decimal)
X_check_train, X_thing = train_test_split(X, test_size=0.33, random_state=101) 
X_check_val, X_check_test = train_test_split(X_thing, test_size=0.5, random_state=42)

## AIF360 Fairness Metrics (Author's Test Data Set)

####Format data

In [19]:
#save datasets at BinaryLabelDatasets

#join X and y data together into an array
train_arr = np.hstack((X_train, y_train.to_numpy().reshape(-1,1)))
val_arr = np.hstack((X_val, y_val.to_numpy().reshape(-1,1)))
test_arr = np.hstack((X_test, y_test.to_numpy().reshape(-1,1)))

#convert back into dataframe
train_df = pd.DataFrame(data=train_arr, columns=heart.columns)
val_df = pd.DataFrame(data=val_arr, columns=heart.columns)
test_df = pd.DataFrame(data=test_arr, columns=heart.columns)

#make sure sex remains binary (this was undone with scaling but 0s remained 0)
train_df.loc[train_df.sex != 0, 'sex'] = 1
val_df.loc[val_df.sex != 0, 'sex'] = 1
test_df.loc[test_df.sex != 0, 'sex'] = 1

#convert sex and target back to int
test_df.sex = test_df.sex.astype(int)
test_df.target = test_df.target.astype(int)

val_df.sex = val_df.sex.astype(int)
val_df.target = val_df.target.astype(int)

train_df.sex = train_df.sex.astype(int)
train_df.target = train_df.target.astype(int)

####Convert in to aif360 Objects

In [20]:
#generate binary label datasets for each of the datasets with the truth value for the target
heart_train_dataset_truth = BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=train_df,
    label_names=['target'],
    protected_attribute_names=['sex'])

heart_val_dataset_truth = BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=val_df,
    label_names=['target'],
    protected_attribute_names=['sex'])

heart_test_dataset_truth = BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=test_df,
    label_names=['target'],
    protected_attribute_names=['sex'])

#save copies of these datasets in order to swap in predictions
heart_train_dataset_preds = heart_train_dataset_truth.copy()
heart_val_dataset_preds = heart_val_dataset_truth.copy()
heart_test_dataset_preds = heart_test_dataset_truth.copy()

####Fit Model

In [21]:
#run this model matching ADS params
model = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5000, 10), random_state=1)
model.fit(X_train, y_train)

#confirm same prediction scores as ADS
print('Val score:',model.score(X_val, y_val))
print('Test score:',model.score(X_test, y_test))

#save preditions on test data
y_preds = model.predict(X_test)

Val score: 0.78
Test score: 0.92


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


####Print Metrics

In [23]:
#save predictions into the dataset
heart_test_dataset_preds.labels = y_preds.reshape(-1,1)

privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

#get classification metrics on test data
metrics = ClassificationMetric(heart_test_dataset_truth, heart_test_dataset_preds,
                      unprivileged_groups=unprivileged_groups,
                      privileged_groups=privileged_groups)

In [24]:
#print comparison metrics
print("Overall Test Accuracy:", metrics.accuracy())
print("Male Test Accuracy:", metrics.accuracy(privileged=True))
print("Female Test Accuracy:", metrics.accuracy(privileged=False))
print('')
print("Test Disparate Impact:", metrics.disparate_impact())
print('')
print("Test FPR:", metrics.false_positive_rate())
print("Test FPR Difference:", metrics.false_positive_rate_difference())
print("")
print("Test FNR:", metrics.false_negative_rate())
print("Test FNR Difference:", metrics.false_negative_rate_difference())
print('')
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_preds))

Overall Test Accuracy: 0.92
Male Test Accuracy: 0.9166666666666666
Female Test Accuracy: 0.9285714285714286

Test Disparate Impact: 1.9285714285714286

Test FPR: 0.15384615384615385
Test FPR Difference: 0.20289855072463767

Test FNR: 0.0
Test FNR Difference: 0.0

Confusion Matrix:
[[22  4]
 [ 0 24]]


####Metric Plots

(we should have some metric plots for the "nutrition label")

####Notes

Notes
- Disparate Impact >1 implies "favorable" outcomes for women
- But in this case, "favorable" means more likely to be diagnosed with heart disease
- DI is Pr(targ=1|women)/Pr(targ=1|men)

Thought:
- SHOULD WE EVALUATE THESE METRICS ON THE VAL AND TEST COMBINED GIVEN THE SIZE OF THE DATASET AND LACK OF HYPERPARAMETER TESTING
- Maybe just throw the val in with the training data and keep the test data as is

##AIF360 Fairness Metrics (Larger Test Set)

We noticed that the author of this model leaves the validation set completely unused.  For a more accurate evaluation of the model, we append the valitation data to the test data and use that for our evaluations.

####Format Data
Combine test and val data into hold data

In [25]:
#use the intermediate held out dataset (test+val) in one dataset
#take the same steps before to get this into a BinLib set

hold_arr = np.hstack((X_hold, y_hold.to_numpy().reshape(-1,1)))
#convert back into dataframe
hold_df = pd.DataFrame(data=hold_arr, columns=heart.columns)
#make sure sex remains binary (this was undone with scaling but 0s remained 0)
hold_df.loc[hold_df.sex != 0, 'sex'] = 1
#convert sex and target back to int
hold_df.sex = hold_df.sex.astype(int)
hold_df.target = hold_df.target.astype(int)

heart_hold_dataset_truth = BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=hold_df,
    label_names=['target'],
    protected_attribute_names=['sex'])

heart_hold_dataset_preds = heart_hold_dataset_truth.copy()
y_preds_new = model.predict(X_hold)

heart_hold_dataset_preds.labels = y_preds_new.reshape(-1,1)

####Convert into air360 Objects

In [26]:
#get classification metrics on bigger test data
metrics_new = ClassificationMetric(heart_hold_dataset_truth, heart_hold_dataset_preds,
                      unprivileged_groups=unprivileged_groups,
                      privileged_groups=privileged_groups)

####Print Metrics

In [27]:
#print comparizon metrics
print("Overall Combined Test Accuracy:", metrics_new.accuracy())
print("Male Combined Test Accuracy:", metrics_new.accuracy(privileged=True))
print("Female Combined Test Accuracy:", metrics_new.accuracy(privileged=False))
print('')
print("Combined Test Disparate Impact:", metrics_new.disparate_impact())
print('')
print("Combined Test FPR:", metrics_new.false_positive_rate())
print("Combined Test FPR Difference:", metrics_new.false_positive_rate_difference())
print("")
print("Combined Test FNR:", metrics_new.false_negative_rate())
print("Combined Test FNR Difference:", metrics_new.false_negative_rate_difference())
print('')
print("Combined Confusion Matrix:")
print(confusion_matrix(y_hold, y_preds_new))

Overall Combined Test Accuracy: 0.85
Male Combined Test Accuracy: 0.821917808219178
Female Combined Test Accuracy: 0.9259259259259259

Combined Test Disparate Impact: 1.828976034858388

Combined Test FPR: 0.20833333333333334
Combined Test FPR Difference: 0.04545454545454544

Combined Test FNR: 0.09615384615384616
Combined Test FNR Difference: -0.0944527736131934

Combined Confusion Matrix:
[[38 10]
 [ 5 47]]


####Plot Metrics

##DisparateImpactRemover Implementation
Here we impleent a simple disparate impact remover algorimth too see how the data is affected.

####Format data and Train DI Remover

In [28]:
#intiates disparate impact repair on data sets
DIR = DisparateImpactRemover(repair_level=1)
train_DIR = DIR.fit_transform(heart_train_dataset_truth)
val_DIR = DIR.fit_transform(heart_val_dataset_truth)
test_DIR = DIR.fit_transform(heart_test_dataset_truth)
hold_DIR = DIR.fit_transform(heart_hold_dataset_truth)

#save copies of these datasets in order to swap in predictions
train_DIR_preds = train_DIR.copy()
val_DIR_preds = val_DIR.copy()
test_DIR_preds = test_DIR.copy()
hold_DIR_preds = hold_DIR.copy()

feature_names = train_DIR.feature_names + ['target']

rep_train_pd = pd.DataFrame(np.hstack([train_DIR.features,train_DIR.labels]),columns=feature_names)
rep_val_pd = pd.DataFrame(np.hstack([val_DIR.features,val_DIR.labels]),columns=feature_names)
rep_test_pd = pd.DataFrame(np.hstack([test_DIR.features,test_DIR.labels]),columns=feature_names)
rep_hold_pd = pd.DataFrame(np.hstack([hold_DIR.features,hold_DIR.labels]),columns=feature_names)

#features
X_train_rep = rep_train_pd[feature_names[:-1]]
X_val_rep = rep_val_pd[feature_names[:-1]]
X_test_rep = rep_test_pd[feature_names[:-1]]
X_hold_rep = rep_hold_pd[feature_names[:-1]]

#labels
y_train_rep = rep_train_pd[feature_names[-1]]
y_val_rep = rep_val_pd[feature_names[-1]]
y_test_rep = rep_test_pd[feature_names[-1]]
y_hold_rep = rep_hold_pd[feature_names[-1]]

####Train Model

In [29]:
model_rep = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5000, 10), random_state=1)

model_rep.fit(X_train_rep,y_train_rep)
#confirm same prediction scores as ADS
print('Hold score:',model_rep.score(X_hold_rep, y_hold_rep))

#save preditions on test data
y_preds_rep = model_rep.predict(X_hold_rep)

Hold score: 0.83


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


####Print Metrics

In [30]:
#save predictions into the dataset
hold_DIR_preds.labels = y_preds_rep.reshape(-1,1)

privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

#get classification metrics on test data
metrics_rep = ClassificationMetric(hold_DIR, hold_DIR_preds,
                      unprivileged_groups=unprivileged_groups,
                      privileged_groups=privileged_groups)

In [31]:
#print comparizon metrics
print("Overall Combined Test Accuracy:", metrics_rep.accuracy())
print("Male Combined Test Accuracy:", metrics_rep.accuracy(privileged=True))
print("Female Combined Test Accuracy:", metrics_rep.accuracy(privileged=False))
print('')
print("Combined Test Disparate Impact:", metrics_rep.disparate_impact())
print('')
print("Combined Test FPR:", metrics_rep.false_positive_rate())
print("Combined Test FPR Difference:", metrics_rep.false_positive_rate_difference())
print("")
print("Combined Test FNR:", metrics_rep.false_negative_rate())
print("Combined Test FNR Difference:", metrics_rep.false_negative_rate_difference())
print('')
print("Combined Confusion Matrix:")
print(confusion_matrix(y_hold_rep, y_preds_rep))

Overall Combined Test Accuracy: 0.83
Male Combined Test Accuracy: 0.7945205479452054
Female Combined Test Accuracy: 0.9259259259259259

Combined Test Disparate Impact: 1.6364522417153997

Combined Test FPR: 0.2708333333333333
Combined Test FPR Difference: -0.022727272727272707

Combined Test FNR: 0.07692307692307693
Combined Test FNR Difference: -0.05997001499250375

Combined Confusion Matrix:
[[35 13]
 [ 4 48]]


####Plot Metrics